In [1]:
import numpy as np
import geopandas as gpd
import pandas as pd
from shapely.geometry import shape
import json
import random
from bokeh.io import output_file, show,output_notebook
from bokeh.models import ColumnDataSource,GeoJSONDataSource,HoverTool,PanTool, WheelZoomTool, BoxSelectTool,GMapPlot, GMapOptions,DataRange1d
from bokeh.models import CategoricalColorMapper,LogColorMapper
from bokeh.plotting import figure
from bokeh.palettes import Viridis6 as palette

In [3]:
phx_crimes = gpd.read_file("phx_crime_counts.shp")

In [4]:
output_notebook()

Loading BokehJS ...

In [5]:
def gpd_bokeh(df):
    """Convert geometries from geopandas to bokeh format"""
    nan = float('nan')
    lons = []
    lats = []
    for i,shape in enumerate(df.geometry.values):
        if shape.geom_type == 'MultiPolygon':
            gx = []
            gy = []
            ng = len(shape.geoms) - 1
            for j,member in enumerate(shape.geoms):
                xy = np.array(list(member.exterior.coords))
                xs = xy[:,0].tolist()
                ys = xy[:,1].tolist()
                gx.extend(xs)
                gy.extend(ys)
                if j < ng:
                    gx.append(nan)
                    gy.append(nan)
            lons.append(gx)
            lats.append(gy)

        else:     
            xy = np.array(list(shape.exterior.coords))
            xs = xy[:,0].tolist()
            ys = xy[:,1].tolist()
            lons.append(xs)
            lats.append(ys) 

    return lons,lats

In [8]:
color_class = [random.randrange(1,25) for _ in range (phx_crimes.shape[0])]#####
lons, lats = gpd_bokeh(phx_crimes)
source = ColumnDataSource(data=dict(
    	x=lons, 
    	y=lats,
        pop = phx_crimes.POP2010,
        crime_den = phx_crimes.crime_dens))

In [9]:
palette.reverse()
color_mapper = LogColorMapper(palette=palette)
TOOLS = "hover,save,pan,box_zoom,wheel_zoom,tap"

In [14]:
map = figure(plot_width=800, plot_height=400,x_axis_location=None,y_axis_location=None,tools=TOOLS)
map.grid.grid_line_color = None
map.patches('x', 'y', source=source, line_color="black", line_width=0.5, fill_color={'field': 'crime_den', 'transform': color_mapper})
map.toolbar_location = None
map.outline_line_color = "white"
map.min_border_top = 0
map.select_one(HoverTool).tooltips = [
    ('Crime_density', '@crime_den'),
    ('PoP2010', '@pop')
    #('class', '@category'),
    #('color','@colorList')
]

In [15]:
show(map)